# Step3: Run the Spark Context 

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('awsproject').getOrCreate()

# Step4: Read the Data

In [3]:
data = spark.read.csv('credit_data.csv',inferSchema=True,
                     header=True)

In [4]:
data.printSchema()

root
 |-- clientid: integer (nullable = true)
 |-- income: double (nullable = true)
 |-- age: double (nullable = true)
 |-- loan: double (nullable = true)
 |-- LTI: double (nullable = true)
 |-- default: integer (nullable = true)



# Step5: Descriptive Analytics

In [5]:
data.describe().show()

+-------+-----------------+------------------+------------------+------------------+--------------------+-------------------+
|summary|         clientid|            income|               age|              loan|                 LTI|            default|
+-------+-----------------+------------------+------------------+------------------+--------------------+-------------------+
|  count|             2000|              2000|              2000|              2000|                2000|               2000|
|   mean|           1000.5|  45331.6000177933|  40.9271432003011| 4444.369694688262| 0.09840282568875686|             0.1415|
| stddev|577.4945887192364|14326.327118850028|13.262449754145178|3045.4100243915227|0.057620211417090464|0.34862375028048487|
|    min|                1|  20014.4894700497|  18.0551885105669|  1.37762959325451| 4.90738078951836E-5|                  0|
|    max|             2000|  69995.6855783239|  63.9717958411202|  13766.0512393337|   0.199937667138875|             

# Step6: Process the data for Mlib Operations

In [6]:
data.columns


['clientid', 'income', 'age', 'loan', 'LTI', 'default']

In [7]:
from pyspark.ml.feature import VectorAssembler

In [8]:
assembler = VectorAssembler(inputCols=['income','age','loan', 'LTI'],outputCol='features')

In [9]:
output = assembler.transform(data)

In [10]:
final_data = output.select('features','default')

# Step7: Divide data into Training and Testing

In [11]:
train_default,test_default = final_data.randomSplit([0.67,0.33])

# Step8: Train the model using Logistic Regression

In [13]:
from pyspark.ml.classification import LogisticRegression
lr_default = LogisticRegression(labelCol='default')

In [14]:
fitted_default_model = lr_default.fit(train_default)

# Step9: Get the Model Summary & Predictions

In [15]:
training_sum = fitted_default_model.summary

In [16]:
training_sum.predictions.describe().show()

+-------+------------------+-------------------+
|summary|           default|         prediction|
+-------+------------------+-------------------+
|  count|              1351|               1351|
|   mean|0.1376757957068838|0.13249444855662473|
| stddev|0.3446869776935242|  0.339153077760388|
|    min|               0.0|                0.0|
|    max|               1.0|                1.0|
+-------+------------------+-------------------+



# Step10: Evaluate the Model

In [17]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [18]:
pred_and_labels = fitted_default_model.evaluate(test_default)

In [19]:
pred_and_labels.predictions.show()


+--------------------+-------+--------------------+--------------------+----------+
|            features|default|       rawPrediction|         probability|prediction|
+--------------------+-------+--------------------+--------------------+----------+
|[20111.3632577471...|      0|[13.2825407971540...|[0.99999829601767...|       0.0|
|[20126.4137681574...|      0|[7.94550620458649...|[0.99964587492654...|       0.0|
|[20155.7923620567...|      0|[2.26702081325889...|[0.90610863760989...|       0.0|
|[20258.538662456,...|      1|[0.14416995466135...|[0.53598018962922...|       0.0|
|[20346.4690526016...|      0|[10.6369953705955...|[0.99997598950139...|       0.0|
|[20436.3312851638...|      0|[12.4367050126816...|[0.99999602985914...|       0.0|
|[20511.4294355735...|      0|[6.37373347234289...|[0.99829712588229...|       0.0|
|[20542.3650727607...|      0|[11.3259853673537...|[0.99998794459466...|       0.0|
|[20568.8913050108...|      0|[0.95140543147374...|[0.72139773379120...|    

In [20]:
default_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                           labelCol='default')

In [21]:
auc = default_eval.evaluate(pred_and_labels.predictions)

In [22]:
auc

0.920934184969371

# Step11: Deploy for new unlabeled data of customers 

In [23]:
new_customers = spark.read.csv('new_customers.csv',inferSchema=True,
                              header=True)

In [24]:
final_lr_model = lr_default.fit(final_data)

In [25]:
new_customers.printSchema()

root
 |-- clientid: integer (nullable = true)
 |-- income: double (nullable = true)
 |-- age: double (nullable = true)
 |-- loan: double (nullable = true)
 |-- LTI: double (nullable = true)



In [26]:
test_new_customers = assembler.transform(new_customers)

In [27]:
final_results = final_lr_model.transform(test_new_customers)

In [28]:
final_results.select('clientid','prediction').show()

+--------+----------+
|clientid|prediction|
+--------+----------+
|    1981|       0.0|
|    1982|       0.0|
|    1983|       0.0|
|    1984|       0.0|
|    1985|       0.0|
|    1986|       0.0|
|    1987|       0.0|
|    1988|       1.0|
|    1989|       0.0|
|    1990|       1.0|
|    1991|       0.0|
|    1992|       0.0|
|    1993|       0.0|
|    1994|       0.0|
|    1995|       0.0|
|    1996|       0.0|
|    1997|       0.0|
|    1998|       0.0|
|    1999|       0.0|
|    2000|       0.0|
+--------+----------+

